# Исследование объявлений о продаже квартир

В вашем распоряжении данные сервиса Яндекс.Недвижимость — архив объявлений о продаже квартир в Санкт-Петербурге и соседних населённых пунктов за несколько лет. Нужно определить рыночную стоимость объектов недвижимости. Ваша задача — установить параметры. Это позволит построить автоматизированную систему: она отследит аномалии и мошенническую деятельность. 

По каждой квартире на продажу доступны два вида данных. Первые вписаны пользователем, вторые — получены автоматически на основе картографических данных. Например, расстояние до центра, аэропорта, ближайшего парка и водоёма. 

**Описание данных**

    airports_nearest — расстояние до ближайшего аэропорта в метрах (м)
    balcony — число балконов
    ceiling_height — высота потолков (м)
    cityCenters_nearest — расстояние до центра города (м)
    days_exposition — сколько дней было размещено объявление (от публикации до снятия)
    first_day_exposition — дата публикации
    floor — этаж
    floors_total — всего этажей в доме
    is_apartment — апартаменты (булев тип)
    kitchen_area — площадь кухни в квадратных метрах (м²)
    last_price — цена на момент снятия с публикации
    living_area — жилая площадь в квадратных метрах(м²)
    locality_name — название населённого пункта
    open_plan — свободная планировка (булев тип)
    parks_around3000 — число парков в радиусе 3 км
    parks_nearest — расстояние до ближайшего парка (м)
    ponds_around3000 — число водоёмов в радиусе 3 км
    ponds_nearest — расстояние до ближайшего водоёма (м)
    rooms — число комнат
    studio — квартира-студия (булев тип)
    total_area — площадь квартиры в квадратных метрах (м²)
    total_images — число фотографий квартиры в объявлении
    
Пояснение: апартаменты — это нежилые помещения, не относящиеся к жилому фонду, но имеющие необходимые условия для проживания.

### Шаг 1. Откроем файл с данными и изучим общую информацию. 

In [ ]:
import seaborn
import matplotlib.pyplot as plt

In [ ]:
import pandas as pd
from IPython.display import display

flat_data = pd.read_csv(r'C:\Users\Айболит\Desktop\DataFrames\projects\недвижимость\real_estate_data.csv', sep='\t', decimal='.')
pd.options.display.max_columns = None
display(flat_data.head(5))

In [ ]:
flat_data.info()

In [ ]:
flat_data.duplicated().sum()

In [ ]:
flat_data.columns = ['total_images', 'last_price', 'total_area', 'first_day_exposition', 'rooms', 'ceiling_height', 'floors_total', 'living_area', 'floor', 'is_apartment', 'studio', 'open_plan', 'kitchen_area', 'balcony', 'locality_name', 'airports_nearest', 'city_centers_nearest', 'parks_around_3000', 'parks_nearest', 'ponds_around_3000', 'ponds_nearest', 'days_exposition']
display(flat_data.head(5))

In [ ]:
flat_data['balcony'].value_counts()

In [ ]:
flat_data['locality_name'].isnull().sum()

In [ ]:
flat_data.dropna(subset = ['locality_name'],inplace = True)
flat_data.info()

Удалил строки с пропущенными данными столбца 'locality_name', не зная к какому населённому пункту они прикреплены, мы не можем ими пользоваться, этих строк - 49, искажений данных не будет. Я надеюсь.... Да не, точно не будет.

In [ ]:
balcony = flat_data['balcony']

In [ ]:
balcony = balcony.fillna(0)


In [ ]:
flat_data['balcony'] = balcony

In [ ]:
flat_data['balcony'] = flat_data['balcony'].astype('int64')


### Вывод

Посмотрел на таблицу. Много пропусков, то, что ясно сразу - удалил и заполнил. На данном этапе пока не однозначно ясно, какие столбцы понадобятся точно, а какие нет.
Дубликаты не обнаружены.

Пропуски могли появиться из-за неполноты предоставленной информации владельцами.

### Шаг 2. Предобработка данных

#### Категоризация типа населённого пункта

In [ ]:
flat_data['locality_name'].value_counts()

In [ ]:
df = flat_data['locality_name'].value_counts()
pd.set_option('display.max_rows', df.shape[0]+1)
#print(df)

In [ ]:
from pymystem3 import Mystem
m = Mystem()

In [ ]:
dictionary = ['товарищество','садовый']
def locality_type_func(row):
    lemma = m.lemmatize(row)
    for word in lemma:
        if word == 'санкт-петербург':
            return(0)
        if word == 'село':
            return(1)
        if word == 'деревня':
            return(2)
        if word == 'поселок':
            for next_word in lemma:
                if next_word == 'городской':
                    return(4)
            return(3)
        if word == 'городской':
            return(4)
        if word in dictionary:
            return(5)
        return(6)

In [ ]:
flat_data['locality_type'] = flat_data['locality_name'].apply(locality_type_func)
flat_data.head()

In [ ]:
flat_data['locality_type'].value_counts()

#### Обработка пропусков в данных

In [ ]:
floor_nan = flat_data.query('floors_total == "NaN"')
floor_nan

тут я посмотрел расположение домов с пропуском в 'floors_total', если бы все были в поселках, а 'floor' == 1, то можно было бы подумать, что это частные дома. Но нет приявязки к локализации и у почти всех этаж выше первого. Значит заменяем пропуски, благо их не много, искажений не должно последовать. Далее смотрю на медиану по 'locality_type' и заменяю пропуски.

In [ ]:
flat_data.dropna(subset = ['floors_total'],inplace = True)
flat_data.info()

я принял сложное решение - не заполнять пропуски в этажности дома, а удалить. Это решение далось тяжко, несколько недель... пытался заменить пропуски на медиану в зависимости от нас.пункта, но порой тогда выходило, что этаж квартиры был выше дома, решил, что нужно в таких случаях прировнять этажность к этажу квартиры или посмотреть самый высокий этаж по нас.пункту и заменить на него. В итоге это всё не объективно, а данных с пропуском в этажности едва сотня.

#### заполнение пропусков касающихся жилой площади и  кухни

я решил, что пропуски в площади кухни следует заполнить медианой исходя из количества комнат,т.е. со схожей планировкой.

In [ ]:
area = flat_data[['kitchen_area', 'total_area', 'rooms']]
area.info()

In [ ]:
kitchen = area.groupby(['rooms'])['kitchen_area'].median()
kitchen

In [ ]:
flat_data.loc[flat_data.loc[:,'rooms'] == 0]

площадь кухни при количестве комнат = 0 везде имеет пропуск. Средняя площадь таких квартир = 25-30 кв.м., это скорее всего студии, при просмотре среза - там только 4 квартиры с площадью больше 40кв.м. : 50кв.м., 73кв.м., 98 кв.м., 371кв.м.  Слабо верится, что помещение 300+ кв.м. имеет 0 комнат, но это помещение со свободной планировкой. А квартиры 50,70+,90+ кв.м. - это студии, где кухня, если есть, плавно переходит в жилое помещение, либо объединена с комнатой. Поэтому заполню пропуски единицей, чтобы при арифметических действиях не получить 0.

In [ ]:
area.loc[area.loc[:,'rooms'] == 0] = area.loc[area.loc[:,'rooms'] == 0].fillna(1)
area.loc[area.loc[:,'rooms'] == 1] = area.loc[area.loc[:,'rooms'] == 1].fillna(9)
area.loc[area.loc[:,'rooms'] == 2] = area.loc[area.loc[:,'rooms'] == 2].fillna(8.7)
area.loc[area.loc[:,'rooms'] == 3] = area.loc[area.loc[:,'rooms'] == 3].fillna(9.5)
area.loc[area.loc[:,'rooms'] == 4] = area.loc[area.loc[:,'rooms'] == 4].fillna(11)
area.loc[area.loc[:,'rooms'] == 5] = area.loc[area.loc[:,'rooms'] == 5].fillna(15.1)
area.loc[area.loc[:,'rooms'] == 6] = area.loc[area.loc[:,'rooms'] == 6].fillna(19.75)
area.loc[area.loc[:,'rooms'] == 7] = area.loc[area.loc[:,'rooms'] == 7].fillna(20)
area.loc[area.loc[:,'rooms'] == 8] = area.loc[area.loc[:,'rooms'] == 8].fillna(25)
area.loc[area.loc[:,'rooms'] == 9] = area.loc[area.loc[:,'rooms'] == 9].fillna(14.3)
area.loc[area.loc[:,'rooms'] == 10] = area.loc[area.loc[:,'rooms'] == 10].fillna(19.8)
area.loc[area.loc[:,'rooms'] == 11] = area.loc[area.loc[:,'rooms'] == 11].fillna(12.6)
area.loc[area.loc[:,'rooms'] == 12] = area.loc[area.loc[:,'rooms'] == 12].fillna(112)
area.loc[area.loc[:,'rooms'] == 14] = area.loc[area.loc[:,'rooms'] == 14].fillna(21.25)


In [ ]:
area.info()

In [ ]:
flat_data['kitchen_area'] = area['kitchen_area']
flat_data.info()

In [ ]:
area = flat_data[['kitchen_area', 'total_area', 'living_area']]
area.info()

In [ ]:
area['living_area'] = area['living_area'].fillna(area['total_area'] - area['kitchen_area'])
area.info()

заменил пропуски в жилой зоне разностью общей площади и кухни. Допускаю, что есть погрешность, но ею можно пренебречь.

In [ ]:
flat_data['living_area'] = area['living_area']
flat_data.info()

#### Категоризация в зависимости от этажа.

Ниже функция каатегоризации.

Категории следующие:

        1 - первый этаж
        2 - средний этаж
        3 - поледний этаж

In [ ]:
def what_floor(data):
    if data['floors_total'] == data['floor']:
        return(3)
    if data['floor'] == 1:
        return(1)
    return(2)

In [ ]:
flat_data['floor_type'] = flat_data.apply(what_floor, axis=1)
flat_data.head()

#### Приведение даты в привычный, рабочий вид

In [ ]:
flat_data['first_day_exposition'] = pd.to_datetime(flat_data['first_day_exposition'], format='%Y-%m-%dT%H:%M:%S')
flat_data.head()

теперь мы можем легко вытащить день, месяц и год публикации

#### изменение типа данных

In [ ]:
flat_data.info()

In [ ]:
flat_data['floors_total'] = flat_data['floors_total'].astype('int64')
flat_data['last_price'] = flat_data['last_price'].astype('int64')
flat_data.head()


#### разбираюсь с потолками

In [ ]:
flat_data['ceiling_height'].value_counts().sort_values()

In [ ]:
flat_data.loc[flat_data.loc[:, 'ceiling_height'] > 6]

In [ ]:
flat_data.loc[flat_data.loc[:, 'ceiling_height'] < 2.30]

In [ ]:
flat_data['ceiling_height'].median()

In [ ]:
flat_data.groupby('locality_type')['ceiling_height'].median()

In [ ]:
ceiling = flat_data[['ceiling_height', 'locality_type']]

In [ ]:
ceiling.loc[ceiling.loc[:,'locality_type'] == 0] = ceiling.loc[ceiling.loc[:,'locality_type'] == 0].fillna(2.7)
ceiling.loc[ceiling.loc[:,'locality_type'] == 1] = ceiling.loc[ceiling.loc[:,'locality_type'] == 1].fillna(2.7)
ceiling.loc[ceiling.loc[:,'locality_type'] == 2] = ceiling.loc[ceiling.loc[:,'locality_type'] == 2].fillna(2.7)
ceiling.loc[ceiling.loc[:,'locality_type'] == 3] = ceiling.loc[ceiling.loc[:,'locality_type'] == 3].fillna(2.7)
ceiling.loc[ceiling.loc[:,'locality_type'] == 4] = ceiling.loc[ceiling.loc[:,'locality_type'] == 4].fillna(2.7)
ceiling.loc[ceiling.loc[:,'locality_type'] == 5] = ceiling.loc[ceiling.loc[:,'locality_type'] == 5].fillna(2.7)
ceiling.loc[ceiling.loc[:,'locality_type'] == 6] = ceiling.loc[ceiling.loc[:,'locality_type'] == 6].fillna(2.7)


In [ ]:
ceiling['ceiling_height'].value_counts()

In [ ]:
s = ceiling['ceiling_height']
s.value_counts()

In [ ]:
s=s.where(s<6, 2.65)
s=s.where(s > 2.3, 2.65)
s.value_counts()

In [ ]:
ceiling.loc[:,'ceiling_height'] = s

In [ ]:
ceiling['ceiling_height'].value_counts()

In [ ]:
flat_data['ceiling_height'] = ceiling['ceiling_height']

In [ ]:

flat_data.info()

In [ ]:
flat_data['ceiling_height'].value_counts()

Заполнил пропуски в столбце отражающем высоту потолка, и заменил выбивающиеся значения на общую медиану, исходя из населенного пункта. Все потолки выше 6 метров и ниже 2.3 метров - заменил на медиану, 6 метров - возможно 2х-ярусная квартира.

В этом шаге я выполнил категоризацию типа нас.пункта, чтобы легче было выделить Санкт-Петербург, поселки, и города.

Удалил строки с пропусками в данных указывающие количество этажей в доме, эта критически важная информация, т.к. в последующем она понадобиться для анализа.

Заполнил пропуски в данных указывающие площадь жилую и кухни, заменил пропуски на медиану квартир отобранных по общему количеству комнат, из соображений общности планировки, а значит и одинаковой площади кухни.

Выполнил категоризацию данных указывающих количество этаже в доме.

Изменил тип даты, сделав возможным вытаскивать из нее информацию о годе, месяце и дне недели.

Такжу изменил тип данных столбцов 'floor' and 'last_price', решение очевидное - этаж хранит целые числа, а в цене недвижимости символы до десятков тысяч вовсе не имеют значения.



### Шаг 3. Посчитаем и добавим в таблицу

#### отношение жилой площади к общей и кухни к жилой

In [ ]:
flat_data.info()

In [ ]:
flat_data['living_total'] = round(flat_data['living_area'] / flat_data['total_area'], 2)

In [ ]:
flat_data['kitchen_living'] = round(flat_data['kitchen_area'] / flat_data['living_area'], 2)

In [ ]:
flat_data.head()

#### Стоимость квадратного метра

In [ ]:
flat_data['price_square_meter'] = flat_data['last_price'] // flat_data['total_area']
flat_data.head()

In [ ]:
flat_data['price_square_meter'] = flat_data['price_square_meter'].astype('int')
flat_data.info()

#### добавление колонок отображения для дня недели, месяца и года

In [ ]:
flat_data['year'] = pd.DatetimeIndex(flat_data['first_day_exposition']).year
flat_data['month'] = pd.DatetimeIndex(flat_data['first_day_exposition']).month
flat_data['day'] = pd.DatetimeIndex(flat_data['first_day_exposition']).weekday

In [ ]:
flat_data.head(3)

### Шаг 4. Проведем исследовательский анализ данных:

посмотрим на распределения

In [ ]:
flat_data['total_area'].hist(bins=50, range=(0,200))
plt.title("распределение продаваемых квартир и площади"); 
plt.xlabel("площадь недвижимости, м^2")
plt.ylabel("количество")
plt.show()

на гистограмме видно, что наиболее распространены в продаже квартиры от 30 до 50 кв.м. Основной массив - это квартиры от 30 до 75 кв.м.

In [ ]:
flat_data['rooms'].hist(range=(0,7),bins=20)

максимум приходится на 1- и 2-комнатные квартиры, как раз хорошо сочетаются с информацией полученной выше - это квартиры до 50 кв.м., прибавим сегмент до 75 кв.м. - получим 3-комнатные.

In [ ]:
flat_data['ceiling_height'].hist(range=(1,6), bins=30)

plt.title("распределение высоты потолков в продаваемой недвижимости"); 
plt.xlabel("высоиа потолков, метр")
plt.ylabel("количество")
plt.show()

с потолками - все по стандарту, мансимум приходится на 2.5 - 2.75 метров.

In [ ]:
flat_data['last_price'].hist(range=(10000,20000000), bins=50)

plt.title("распределение стоимости продаваемой недвижимости"); 
plt.xlabel("стоимость")
plt.ylabel("частота")
plt.show()

In [ ]:
flat_data['last_price'].value_counts()

#### 2. Время продажи

In [ ]:
flat_data['days_exposition'].hist(range=(0, 1000))

plt.title("распределение по продолжительности продажи"); 
plt.xlabel("продолжительность, день")
plt.ylabel("колиество")
plt.show()

время продажи у большинства квартир не превышает 100 дней. Давайте посмотрим на первые 100 дней внимательней.

Заметим, что нами считается квартира проданной с момента снятия объявления с доски.

In [ ]:
flat_data['days_exposition'].hist(range=(0, 100))

plt.title("распределение по продолжительности продажи"); 
plt.xlabel("продолжительность, день")
plt.ylabel("колиество")
plt.show()

Пик продаж приходится на 40-50 дни размещения объявления. Также можно заметить, что большинство квартир продается в певые 50 дней.

In [ ]:
flat_data.boxplot(column = 'days_exposition')
plt.title("ящик с усами по времени продажи"); 


In [ ]:
flat_data['days_exposition'].describe()

In [ ]:
time_long = flat_data.query('days_exposition > 500')
time_long.head(10)

In [ ]:
time_fast = flat_data.query('days_exposition < 100')

In [ ]:
time_norm = flat_data.query('days_exposition < 500')

In [ ]:
time_norm.boxplot(column = 'days_exposition')
plt.title('ящик с усами по времени продажи после "отсечения объявлений сроком более 500 дней"'); 


In [ ]:
time_norm['days_exposition'].describe()

In [ ]:
time_long['locality_type'].value_counts()

In [ ]:
time_long['last_price'].describe()

In [ ]:
time_long.query('locality_type == 1')

In [ ]:
time_long.query('days_exposition > 1000')

In [ ]:
time_long['last_price'].mean()


In [ ]:
time_long['last_price'].median()

In [ ]:
time_norm['last_price'].median()

In [ ]:
time_norm['last_price'].mean()

я посмотрел как распределен срок продажи квартир. У кого-то это занимает 1 день, а у кого-то больше 4х лет. Есть пропуски в данных, которые я не стал заполнять, чтобы не исказить данные. Слишком малые или слишком больши данные аналогично, считаю, что не нужно исключать, т.к. нет причин считать их выбросом. Это недвижимость, квартиры, которые долго стоят в продаже имеют цену выше, возможно продавец не хотел снижать цену и ждал своего покупателя. Также и с квартирами, которые быстро ушли с продажи.

Если цена квартиры не выше медианы, но она долго стоит в продаже - она находится в посёлке, соответственно нет желающих её взять по цене, по которой можно приобрести квартиру в городе.

####  Факторы влияющие на стоимость квартиры.

##### 1. Площадь недвижимости

In [ ]:
flat_data.info()

In [ ]:
square = flat_data[['last_price', 'total_area']]

In [ ]:
square.corr()

Далее я компаную квартиры в зависимости от их площади. Чтобы данные были более однородны и, таким образом, оценка полученных наблюдений была более объективна.

In [ ]:
square20 = flat_data.query('total_area <= 20').median()
square30 = flat_data.query('20 < total_area <= 30').median()
square40 = flat_data.query('30 < total_area <= 40').median()
square50 = flat_data.query('40 < total_area <= 50').median()
square60 = flat_data.query('50 < total_area <= 60').median()
square70 = flat_data.query('60 < total_area <= 70').median()
square80 = flat_data.query('70 < total_area <= 80').median()
square90 = flat_data.query('80 < total_area <= 90').median()
square100 = flat_data.query('90 < total_area <= 100').median()
square110 = flat_data.query('100 < total_area <= 110').median()
square120 = flat_data.query('110 < total_area <= 120').median()
square130 = flat_data.query('120 < total_area <= 130').median()
square140 = flat_data.query('130 < total_area <= 140').median()
square150 = flat_data.query('140 < total_area <= 150').median()
square160 = flat_data.query('150 < total_area <= 160').median()
square170 = flat_data.query('160 < total_area <= 170').median()
square180 = flat_data.query('170 < total_area <= 180').median()
square190 = flat_data.query('180 < total_area <= 190').median()
square200 = flat_data.query('190 < total_area <= 200').median()
square100500 = flat_data.query(' total_area > 200').median()

In [ ]:
sq_median = pd.DataFrame({
    'price': [square20['last_price'],square30['last_price'],square40['last_price'],square50['last_price'],square60['last_price'],square70['last_price'],square80['last_price'],square90['last_price'],square100['last_price'],square110['last_price'],square120['last_price'],square130['last_price'],square140['last_price'],square150['last_price'],square160['last_price'],square170['last_price'],square180['last_price'],square190['last_price'],square200['last_price'],square100500['last_price']]}) 
sq_median

In [ ]:
seaborn.regplot(x='total_area', y='last_price', data=square)
plt.title("зависимость площади продаваемой недвижимости и ее цены"); 
plt.xlabel("площадь, кв.метры")
plt.ylabel("стоимость")
plt.show()

In [ ]:
sq_median.plot()

plt.title("зависимость площади продаваемой недвижимости и ее цены, по медиане в зависимости от категории"); 
plt.xlabel("медиана площади, кв.метры")
plt.ylabel("стоимость")
plt.show()

глядя на график, видно, что с ценой растёт и увеличение площади квартиры, также коэф.корреляции цены от площади - 0,65, то есть умеренный. Я взял медиану по цене для промежутков площади, чтобы убрать излишнюю "скученность" графика, которую могут привносить другие факторы, такие как ремонт, расположение, жадность.

##### 2. Количество комнат

забегу вперёд, сказав, что количество комнат крайне тесно коррелирует с площадью недвижимости. 

In [ ]:
room_square = flat_data[['rooms','total_area']]
room_square.corr()

коэф.корреляции указывает на сильную зависимость количества комнат и площади. Значит - что верно для площади, будет верно и для параметра "количество комнат".

In [ ]:
seaborn.regplot(x='rooms', y='total_area', data=room_square)
plt.title("зависимость цены продаваемой недвижимости и количества комнат"); 
plt.xlabel("количество комнат")
plt.ylabel("стоимость")
plt.show()

Видим хорошую корреляцию

In [ ]:
room_price = flat_data.pivot_table(index='rooms',values='last_price',aggfunc='median')
room_price

In [ ]:
room_price.plot(figsize=(7,5))


plt.title("зависимость цены продаваемой недвижимости и количества комнат по медиане"); 
plt.xlabel("количество комнат")
plt.ylabel("стоимость, медиана")
plt.show()

In [ ]:
room_price.query('rooms == 12')

In [ ]:
room_price.info()

In [ ]:
room_price = room_price.loc[room_price['last_price'] != 420000000]

In [ ]:
room_price.plot(figsize=(10,5))
plt.title("зависимость цены продаваемой недвижимости и количества комнат по медиане"); 
plt.xlabel("количество комнат")
plt.ylabel("стоимость, медиана")
plt.show()

мы видим, что с увеличением количества комнат, действительно увеличивается и стоимость. Причем зависимость количества комнат - ценнее, чем общая площадь, зависимость выражена сильнее. Можно добавить, если привязывать стоимость недвижимости к количеству комнат, то идёт резкий подьём с 3х комнат до 7. Т.к. тут наибольшая привязка количеств комнат к площади.

In [ ]:
rooms_area = flat_data.pivot_table(index='rooms',values='total_area',aggfunc='median')
rooms_area

In [ ]:
rooms_area.plot(figsize=(10,5))

plt.title("зависимость цены продаваемой недвижимости и количества комнат по медиане, со всеми объектами без исключения"); 
plt.xlabel("количество комнат")
plt.ylabel("стоимость, медиана")
plt.show()

видя 2 пика на 12 и 15 комнатах, если посмотрим на эти объявления глазом и обратимсяк графику из прошлого шага, то увидем, что квартиры имеют просто невероятную площадь, этим объясняется и пик в стоимости. Графики   зависимости стоимости от количества комнат   и   площади от количества комнат   почти идентичны. Поэтому можно смело сказать, что количество комнат это вторичный фактор, зависящий от площади.

##### 3. Удалённость от центра

я не стал заполнять пропуски в столбце удаленности от центра. Потому что как я могу это сделать? на медиану - не сойдёт, будет искажение данных - 5000 значений из 23565 - почти четверть. Возможно эти квартиры находятся в центре и следовало заполнить нулями, а возможно адрес квартиры не указан был, поэтому значение может быть любым.

In [ ]:
flat_data.info()

In [ ]:
experiment = flat_data[['city_centers_nearest','locality_type','locality_name','last_price']]
center = flat_data[['city_centers_nearest','last_price']]

In [ ]:
experiment = experiment.fillna(0)

In [ ]:
experiment['city_centers_nearest'].value_counts().sort_values()

In [ ]:
experiment.corr()

In [ ]:
center.corr()

In [ ]:
experiment0 = experiment.loc[experiment.loc[:,'city_centers_nearest'] == 0]


In [ ]:
experiment0['locality_type'].value_counts()

In [ ]:
#center.sort_values('city_centers_nearest').plot(y='last_price', x='city_centers_nearest',  style = 'o',  grid = True, figsize = (12, 6))

seaborn.regplot(x='city_centers_nearest', y='last_price', data=center)

plt.title("зависимость цены продаваемой недвижимости от удаленности от центра"); 
plt.xlabel("удаленность от центра, м.")
plt.ylabel("стоимость")
plt.show()

In [ ]:
experiment.sort_values('city_centers_nearest').plot(y='last_price', x='city_centers_nearest',  style = 'o',  grid = True, figsize = (12, 6))

plt.title("зависимость цены продаваемой недвижимости от удаленности от центра"); 
plt.xlabel("удаленность от центра, м.")
plt.ylabel("стоимость")
plt.show()

In [ ]:
center10 = center.query('city_centers_nearest < 10000')

In [ ]:
center10.sort_values('city_centers_nearest').plot(y='last_price', x='city_centers_nearest',  style = 'o',  grid = True, figsize = (12, 6))

plt.title("зависимость цены продаваемой недвижимости от удаленности от центра на удаленности до 10000 м."); 
plt.xlabel("удаленность от центра, м.")
plt.ylabel("стоимость")
plt.show()

на первом графике видно, что, если брать всю удаленность,это от 0 до более, чем 60000 метров, то на отрезке 0-10км. квартиры стоят существенно дороже, чем далее 10 км.
Но если мы приблизим и рассмотрим участок 0-10 км, то увидим, что нет привязки на участке 0-8км. Вероятно это связано, что данные у нас не стандартизованы. Мы не можем оценивать квартиру в 30кв.м. в центре и квартиру 100кв.м. на удалении в 8км от центра. Предлагаю посмотреть как распределена стоимость квартир одной площади, но с разной удалённостью от центра.

Сделаю сейчас то, что нужно было сделать давно

In [ ]:
def square_flat(row):
    if row < 20:
        return(2)
    if (row >= 20) & (row < 30):
        return(3)
    if (row >= 30) & (row < 40):
        return(4)
    if (row >= 40) & (row < 50):
        return(5)
    if (row >= 50) & (row < 60):
        return(6)
    if (row >= 60) & (row < 70):
        return(7)
    if (row >= 70) & (row < 80):
        return(8)
    if (row >= 80) & (row < 90):
        return(9)
    if (row >= 90) & (row < 100):
        return(10)
    if (row >= 100) & (row < 110):
        return(11)
    if (row >= 110) & (row < 120):
        return(12)
    if (row >= 120) & (row < 130):
        return(13)
    if (row >= 130) & (row < 140):
        return(14)
    if (row >= 140) & (row < 150):
        return(15)
    if (row >= 150) & (row < 160):
        return(16)
    if (row >= 160) & (row < 170):
        return(17)
    if (row >= 170) & (row < 180):
        return(18)
    if (row >= 180) & (row < 190):
        return(19)
    if (row >= 190) & (row < 200):
        return(20)
    if row >= 20:
        return(888)

In [ ]:
flat_data['square_type'] = flat_data['total_area'].apply(square_flat)

тут я произвёл категоризацию в зависимости от площади квартиры.

In [ ]:
flat_data.head(5)

In [ ]:
center = flat_data[['last_price','city_centers_nearest','square_type']]

In [ ]:
center20 = center.loc[center.loc[:, 'square_type'] == 2]
seaborn.regplot(x='city_centers_nearest', y='last_price', data=center20)
plt.title("зависимость цены продаваемой недвижимости площадью до 20 кв.м. от удаленности от центра"); 
plt.xlabel("удаленность от центра, м.")
plt.ylabel("стоимость")

In [ ]:
center30 = center.loc[center.loc[:, 'square_type'] == 3]
seaborn.regplot(x='city_centers_nearest', y='last_price', data=center30)
plt.title("зависимость цены продаваемой недвижимости площадью до 20-30 кв.м. от удаленности от центра"); 
plt.xlabel("удаленность от центра, м.")
plt.ylabel("стоимость")


In [ ]:
center30 = center30.query('city_centers_nearest <20000')
seaborn.regplot(x='city_centers_nearest', y='last_price', data=center30)
plt.title("зависимость цены продаваемой недвижимости площадью до 20 кв.м. от удаленности от центра в диапозоне до 20км."); 
plt.xlabel("удаленность от центра, м.")
plt.ylabel("стоимость")

In [ ]:
center40 = center.loc[center.loc[:, 'square_type'] == 4]
seaborn.regplot(x='city_centers_nearest', y='last_price', data=center40)
plt.title("зависимость цены продаваемой недвижимости площадью 30-40 кв.м. от удаленности от центра"); 
plt.xlabel("удаленность от центра, м.")
plt.ylabel("стоимость")

In [ ]:
center50 = center.loc[center.loc[:, 'square_type'] == 5]
seaborn.regplot(x='city_centers_nearest', y='last_price', data=center50)
plt.title("зависимость цены продаваемой недвижимости площадью 40-50 кв.м. от удаленности от центра"); 
plt.xlabel("удаленность от центра, м.")
plt.ylabel("стоимость")

In [ ]:
center60 = center.loc[center.loc[:, 'square_type'] == 6]
seaborn.regplot(x='city_centers_nearest', y='last_price', data=center60)
plt.title("зависимость цены продаваемой недвижимости площадью 50-60 кв.м. от удаленности от центра"); 
plt.xlabel("удаленность от центра, м.")
plt.ylabel("стоимость")

In [ ]:
center70 = center.loc[center.loc[:, 'square_type'] == 7]
seaborn.regplot(x='city_centers_nearest', y='last_price', data=center70)
plt.title("зависимость цены продаваемой недвижимости площадью 60-70 кв.м. от удаленности от центра"); 
plt.xlabel("удаленность от центра, м.")
plt.ylabel("стоимость")

In [ ]:
center80 = center.loc[center.loc[:, 'square_type'] == 8]
seaborn.regplot(x='city_centers_nearest', y='last_price', data=center80)
plt.title("зависимость цены продаваемой недвижимости площадью 70-80 кв.м. от удаленности от центра"); 
plt.xlabel("удаленность от центра, м.")
plt.ylabel("стоимость")

In [ ]:
center90 = center.loc[center.loc[:, 'square_type'] == 9]
seaborn.regplot(x='city_centers_nearest', y='last_price', data=center90)
plt.title("зависимость цены продаваемой недвижимости площадью 80-90 кв.м. от удаленности от центра"); 
plt.xlabel("удаленность от центра, м.")
plt.ylabel("стоимость")

In [ ]:
center100 = center.loc[center.loc[:, 'square_type'] == 10]
seaborn.regplot(x='city_centers_nearest', y='last_price', data=center100)
plt.title("зависимость цены продаваемой недвижимости площадью 90-100 кв.м. от удаленности от центра"); 
plt.xlabel("удаленность от центра, м.")
plt.ylabel("стоимость")

я дальше не стал строить графики, т.к. все они типичны и повторяют друг друга. Имеют вид прямоугольной трапеции с линейной хависимостью к уменьшению цены при отдалении от центра. Теперь мы можем сказать, что - да, близость к центру влияет на стоимость квартиры, точнее - тенденция к её увеличению, ибо в центре квартиры стоят дороже, чем дальше от него с одинаковой площадью. 

#####  4. Влияние этажа на стоимость квартиры

In [ ]:
flat_data.info()

In [ ]:
floor_df = flat_data[['floor_type','last_price','square_type']]

In [ ]:
floor_df.corr()

На первый взгляд зависимость слабая, но давайте посмотрим внимательней, объединим квартиры по одной площади и расстоянию к центру.

In [ ]:
floor_df = flat_data[['floor_type','last_price','square_type', 'city_centers_nearest']]

In [ ]:
standart1 = floor_df.query('(square_type == 2) & (city_centers_nearest < 5000)')

In [ ]:
standart1.corr()

Интересно. Сразу кардинально сменилась картина. Было 0,05, а стало 0,74. Давайте посмотрим на график.

In [ ]:
standart1 = standart1[['floor_type','last_price']]
seaborn.regplot(x='floor_type', y='last_price', data=standart1)
plt.title("зависимость цены продаваемой недвижимости площадью до 20кв.м. от этажа при диапозоне удаленности от центра до 5км."); 
plt.xlabel("категория в зависимости от этажа")
plt.ylabel("стоимость")

In [ ]:
standart1.loc[standart1.loc[:,'floor_type'] == 3]

не совсем чистые данные, нет последнего этажа и данных не много. Посмотрим, что по другим категориям.

In [ ]:
standart3 = floor_df.query('(square_type == 3) & (city_centers_nearest < 5000)')
standart3.corr()

In [ ]:
standart3 = standart3[['floor_type','last_price']]
seaborn.regplot(x='floor_type', y='last_price', data=standart3)
plt.title("зависимость цены продаваемой недвижимости площадью 20-30кв.м. от этажа при диапозоне удаленности от центра до 5км."); 
plt.xlabel("категория в зависимости от этажа")
plt.ylabel("стоимость")

In [ ]:
standart4 = floor_df.query('(square_type == 4) & (city_centers_nearest < 5000)')
standart4.corr()

In [ ]:
standart4 = standart4[['floor_type','last_price']]
seaborn.regplot(x='floor_type', y='last_price', data=standart4)
plt.title("зависимость цены продаваемой недвижимости площадью 30-40кв.м. этажа  при диапозоне удаленности от центра до 5км."); 
plt.xlabel("категория в зависимости от этажа")
plt.ylabel("стоимость")

In [ ]:
standart5 = floor_df.query('(square_type == 5) & (city_centers_nearest < 5000)')
standart5.corr()

In [ ]:
standart5 = standart5[['floor_type','last_price']]
seaborn.regplot(x='floor_type', y='last_price', data=standart5)
plt.title("зависимость цены продаваемой недвижимости площадью 40-50кв.м. от этажа  при диапозоне удаленности от центра до 5км."); 
plt.xlabel("категория в зависимости от этажа")
plt.ylabel("стоимость")

In [ ]:
standart6 = floor_df.query('(square_type == 6) & (city_centers_nearest < 5000)')
standart6.corr()

In [ ]:
standart6 = standart6[['floor_type','last_price']]
seaborn.regplot(x='floor_type', y='last_price', data=standart6)
plt.title("зависимость цены продаваемой недвижимости площадью 50-60кв.м. от этажа  при диапозоне удаленности от центра до 5км."); 
plt.xlabel("категория в зависимости от этажа")
plt.ylabel("стоимость")

In [ ]:
standart7 = floor_df.query('(square_type == 7) & (city_centers_nearest < 5000)')
standart7.corr()

In [ ]:
standart7 = standart7[['floor_type','last_price']]
seaborn.regplot(x='floor_type', y='last_price', data=standart7)
plt.title("зависимость цены продаваемой недвижимости площадью 60-70кв.м. от этажа  при диапозоне удаленности от центра до 5км."); 
plt.xlabel("категория в зависимости от этажа")
plt.ylabel("стоимость")

In [ ]:
standart8 = floor_df.query('(square_type == 8) & (city_centers_nearest < 5000)')
standart8.corr()

In [ ]:
standart8 = standart8[['floor_type','last_price']]
seaborn.regplot(x='floor_type', y='last_price', data=standart8)
plt.title("зависимость цены продаваемой недвижимости площадью 70-80кв.м. от этажа  при диапозоне удаленности от центра до 5км."); 
plt.xlabel("категория в зависимости от этажа")
plt.ylabel("стоимость")

In [ ]:
standart9 = floor_df.query('(square_type == 9) & (city_centers_nearest < 5000)')
standart9.corr()

In [ ]:
standart9 = standart9[['floor_type','last_price']]
seaborn.regplot(x='floor_type', y='last_price', data=standart9)
plt.title("зависимость цены продаваемой недвижимости площадью 80-90кв.м. от этажа  при диапозоне удаленности от центра до 5км."); 
plt.xlabel("категория в зависимости от этажа")
plt.ylabel("стоимость")

In [ ]:
standart10 = floor_df.query('(square_type == 10) & (city_centers_nearest < 5000)')
standart10.corr()

In [ ]:
standart10 = standart10[['floor_type','last_price']]
seaborn.regplot(x='floor_type', y='last_price', data=standart10)
plt.title("зависимость цены продаваемой недвижимости площадью 90-100кв.м. от этажа  при диапозоне удаленности от центра до 5км."); 
plt.xlabel("категория в зависимости от этажа")
plt.ylabel("стоимость")

In [ ]:
standart11 = floor_df.query('(square_type == 11) & (city_centers_nearest < 5000)')
standart11.corr()

In [ ]:
standart11 = standart11[['floor_type','last_price']]
seaborn.regplot(x='floor_type', y='last_price', data=standart11)
plt.title("зависимость цены продаваемой недвижимости площадью 100-110кв.м. от этажа  при диапозоне удаленности от центра до 5км."); 
plt.xlabel("категория в зависимости от этажа")
plt.ylabel("стоимость")

In [ ]:
standart12 = floor_df.query('(square_type == 12) & (city_centers_nearest < 5000)')
standart12.corr()

In [ ]:
standart12 = standart12[['floor_type','last_price']]
seaborn.regplot(x='floor_type', y='last_price', data=standart12)
plt.title("зависимость цены продаваемой недвижимости площадью 110-120кв.м. от этажа  при диапозоне удаленности от центра до 5км."); 
plt.xlabel("категория в зависимости от этажа")
plt.ylabel("стоимость")

Логичная мысль при знакомстве с графиками подтвердилась - стоимость квартир схожей площади и отдаленности от центра ниже, если этаж первый, стоимость среднего этажа всегда выше, чем первого и последнего.

In [ ]:
df = standart12.loc[standart12.loc[:, 'floor_type'] == 3].max()
df

In [ ]:
standart12 = standart12.loc[standart12.loc[:, 'last_price'] != 39000000]

In [ ]:
df = standart12.loc[standart12.loc[:, 'floor_type'] == 1].max()
df

In [ ]:
standart12 = standart12.loc[standart12.loc[:, 'last_price'] != 22400000]

In [ ]:
standart12.plot(y='floor_type', x='last_price',  style = 'o',  grid = True, figsize = (10, 5))

In [ ]:
standart12.corr()

Увы, но метод corr() упорно отказывается замечать зависимость.

In [ ]:
standart_all = floor_df.query('(square_type > 12) & (city_centers_nearest < 5000)')
standart_all.corr()

In [ ]:
standart_all.plot(y='floor_type', x='last_price',  style = 'o',  grid = True, figsize = (10, 5))

In [ ]:
floor_df = floor_df[['floor_type','last_price']]
floor_df.plot(y='floor_type', x='last_price',  style = 'o',  grid = True, figsize = (10, 5))

Метод corr() не выявил чёткой закономерности стоимости квартир от этажа. Но на графиках мы видим, что средний этаж (2.00) более смещен к в сторону роста цены. Но стоит помнить, что "средних" квартир больше. Т.к. график имеет тенденцию к повышению цены у средних этажей, то можно сказать, что да, этаж влияет на формирование цены, но не так критично, как площадь.

##### 5. Поиск зависимости стоимости от даты размещения.

может ли зависеть стоимость недвижимости от даты публикации объявления? Предположение - наверно может, если дата связана с некоторыми событиями в стране и мире. Все мы знаем, что спрос рождает предложение. Если есть увеличение спроса в те или иные временные периоды, логично предположить, что цена повысится, это работат и в обратном порядке. Давайте попробуем выделить временные промежутки, когда было получено много предложений по продаже квартир/мало объявлений и сравним медианы цен примерно "одинаковых" квартир по площади и удалённости от центра.

In [ ]:
flat_data.info()

In [ ]:
time_sell = flat_data[['square_type', 'year','month','day','last_price', 'first_day_exposition', 'days_exposition']]

In [ ]:
time_sell.corr()

Давайте узнаем - какие месяца самые любимые для продающих недвижимость.

In [ ]:
time_sell['month'].value_counts() 

Глядя на общее распределение с 14 по неполный 19 гг., можно сказать следующее - предложение начинает расти с сентября, в декабре и январе - резкий спад, оно и понятно, все празднуют и радуются жизни, снова достигает пика к февралю и начинает по-немногу снижаться. Летом все заняты отпусками.

In [ ]:
date = time_sell.pivot_table(index=['year','month'], values='day',  aggfunc='count')
table = date.reset_index()
i = 0
while i < 54:
    table.loc[i,'month'] = i
    i+=1
table = table[['month','day']]
date.reset_index()

In [ ]:
table.plot(x='month', y='day', figsize=(10,8), grid=True)
plt.title("зависимость цены продаваемой недвижимости по времени выставления объявления"); 
plt.xlabel("временная шкала")
plt.ylabel("стоимость")

Распределение по годам - в общем подтверждает отраженное ранее. Летом, если появляются новые объявления, то не значительно.
Мы видим, что был резкий взлёт продаваемости с мая 2017 года, с чем он был связан, возможно с рекламой сайта, возможно со сдачей высоток, тогда нужно посмотреть на квартиры выставленные в мае 2017. Кривая уходит глубоко вниз, возвращаясь к значениям 14-15гг. и вновь поступают предложения. Тут возможно несколько исходов - проблемы с сайтом и все объявления слетели, либо квартиры были распроданы, а новые предложения не поступали. Интересно посмотреть на цены на вершинах графика и внизу.

In [ ]:
plato1 = flat_data.query('(year <= 2016) & (month < 2)')
pic1 = flat_data.query('(year == 2016) & (2 < month < 7)')
plato2 = flat_data.query('(year == 2016) & (7 <= month <= 12)')
pic2 = flat_data.query('(year == 2017) & (4 <= month <= 10)')
mega_pic1 = flat_data.query('(year == 2017) & (month == 11)')
plato3_1 = flat_data.query('(year == 2017) & (month == 12)')
plato3_2 = flat_data.query('(year == 2018) & (month == 1)')
join_plato = [plato3_1, plato3_2]
plato3 = pd.concat(join_plato)
mega_pic2 = flat_data.query('(year == 2017) & (month == 11)')
crizis = flat_data.query('(year == 2018) & (4 <= month <= 6)')

In [ ]:
plato1['locality_type'].value_counts()

In [ ]:
pic1['locality_type'].value_counts()

видим увеличение количества объявлений в 10 раз во всех типах нас.пунктов, скорее всего реклама сайта...

In [ ]:
plato2['locality_type'].value_counts()

In [ ]:
pic2['locality_type'].value_counts()

In [ ]:
mega_pic1['locality_type'].value_counts()

In [ ]:
mega_pic2['locality_type'].value_counts()

In [ ]:
plato2['locality_type'].value_counts()

In [ ]:
plato3['locality_type'].value_counts()

In [ ]:
crizis['locality_type'].value_counts()

In [ ]:
median_price = plato1.pivot_table(index=['square_type'], values='last_price', aggfunc='median')
median_price = median_price.reset_index()
median_price.columns = ['square_type','plato1']

In [ ]:
median_price_pic1 = pic1.pivot_table(index=['square_type'], values='last_price', aggfunc='median')
median_price_pic1 = median_price_pic1.reset_index()
median_price_pic1.columns = ['square_type','pic1']

In [ ]:
median_price_plato2 = plato2.pivot_table(index=['square_type'], values='last_price', aggfunc='median')
median_price_plato2 = median_price_plato2.reset_index()
median_price_plato2.columns = ['square_type','plato2']

median_price_pic2 = pic2.pivot_table(index=['square_type'], values='last_price', aggfunc='median')
median_price_pic2 = median_price_pic2.reset_index()
median_price_pic2.columns = ['square_type','pic2']

median_price_mega_pic1 = mega_pic1.pivot_table(index=['square_type'], values='last_price', aggfunc='median')
median_price_mega_pic1 = median_price_mega_pic1.reset_index()
median_price_mega_pic1.columns = ['square_type','mega_pic1']

median_price_plato3 = plato3.pivot_table(index=['square_type'], values='last_price', aggfunc='median')
median_price_plato3 = median_price_plato3.reset_index()
median_price_plato3.columns = ['square_type','plato3']

median_price_mega_pic2 = mega_pic2.pivot_table(index=['square_type'], values='last_price', aggfunc='median')
median_price_mega_pic2 = median_price_mega_pic2.reset_index()
median_price_mega_pic2.columns = ['square_type','mega_pic2']

median_price_crizis = crizis.pivot_table(index=['square_type'], values='last_price', aggfunc='median')
median_price_crizis = median_price_crizis.reset_index()
median_price_crizis.columns = ['square_type','crizis']

In [ ]:
plato1_pic1 = median_price.merge(median_price_pic1, on='square_type', how='right')
general = plato1_pic1.merge(median_price_plato2, on='square_type', how='right')
general = general.merge(median_price_pic2, on='square_type', how='right')
general = general.merge(median_price_mega_pic1, on='square_type', how='right')
general = general.merge(median_price_plato3, on='square_type', how='right')
general = general.merge(median_price_mega_pic2, on='square_type', how='right')
general = general.merge(median_price_crizis, on='square_type', how='right')
general

In [ ]:
general.head(20)

далее решил посмотреть также как будет выглядеть картина, если мы будем работать не с медианой, а со средним, для этого хочу удалить выбивающиеся из общего числа значения.

In [ ]:
st2 = flat_data.query('square_type == 2')
st2.boxplot(column =['last_price'])

In [ ]:
st2 = st2.loc[st2.loc[:, 'last_price'] < 300000]

In [ ]:
st3 = flat_data.query('square_type == 3')
st3.boxplot(column =['last_price'])

In [ ]:
st3['last_price'].describe()

In [ ]:
st3 = st3.loc[st3.loc[:, 'last_price'] > 900000]
st3 = st3.loc[st3.loc[:, 'last_price'] < 5600000]

In [ ]:
st4 = flat_data.query('square_type == 4')
st4.boxplot(column =['last_price'])

In [ ]:
st4['last_price'].describe()

In [ ]:
st4 = st4.loc[st4.loc[:, 'last_price'] >= 2780000]
st4 = st4.loc[st4.loc[:, 'last_price'] <= 3950000]

In [ ]:
st5 = flat_data.query('square_type == 5')
st5['last_price'].describe()

In [ ]:
st5 = st5.loc[st5.loc[:, 'last_price'] >= 3225000]
st5 = st5.loc[st5.loc[:, 'last_price'] <= 4749000]

In [ ]:
st6 = flat_data.query('square_type == 6')
st6['last_price'].describe()

In [ ]:
st6 = st6.loc[st6.loc[:, 'last_price'] >= 3900000]
st6 = st6.loc[st6.loc[:, 'last_price'] <= 5800000]

In [ ]:
st7 = flat_data.query('square_type == 7')
st7['last_price'].describe()

In [ ]:
st7 = st7.loc[st7.loc[:, 'last_price'] >= 4600000]
st7 = st7.loc[st7.loc[:, 'last_price'] <= 7160000]

In [ ]:
st8 = flat_data.query('square_type == 8')
st8['last_price'].describe()

In [ ]:
st8 = st8.loc[st8.loc[:, 'last_price'] >= 5400000]
st8 = st8.loc[st8.loc[:, 'last_price'] <= 8446750]

In [ ]:
st9 = flat_data.query('square_type == 9')
st9['last_price'].describe()

In [ ]:
st9 = st9.loc[st9.loc[:, 'last_price'] >= 6450000]
st9 = st9.loc[st9.loc[:, 'last_price'] <= 10300000]

In [ ]:
st10 = flat_data.query('square_type == 10')
st10['last_price'].describe()

In [ ]:
st10 = st10.loc[st10.loc[:, 'last_price'] >= 7800000]
st10 = st10.loc[st10.loc[:, 'last_price'] <= 12600000]

In [ ]:
st11 = flat_data.query('square_type == 11')
st11['last_price'].describe()

In [ ]:
st11 = st11.loc[st11.loc[:, 'last_price'] >= 9313558]
st11 = st11.loc[st11.loc[:, 'last_price'] <= 14500000]

In [ ]:
st12 = flat_data.query('square_type == 12')
st12['last_price'].describe()

In [ ]:
st12 = st12.loc[st12.loc[:, 'last_price'] >= 10000000]
st12 = st12.loc[st12.loc[:, 'last_price'] <= 17388750]

In [ ]:
st13 = flat_data.query('square_type == 13')
st13['last_price'].describe()

In [ ]:
st13 = st13.loc[st13.loc[:, 'last_price'] >= 11525000]
st13 = st13.loc[st13.loc[:, 'last_price'] <= 20900000]

In [ ]:
st14 = flat_data.query('square_type == 14')
st14['last_price'].describe()

In [ ]:
st14 = st14.loc[st14.loc[:, 'last_price'] >= 12974250]
st14 = st14.loc[st14.loc[:, 'last_price'] <= 22432310]

In [ ]:
st15 = flat_data.query('square_type == 15')
st15['last_price'].describe()

In [ ]:
st15 = st15.loc[st15.loc[:, 'last_price'] >= 13000000]
st15 = st15.loc[st15.loc[:, 'last_price'] <= 25000000]

In [ ]:
st16 = flat_data.query('square_type == 16')
st16['last_price'].describe()

In [ ]:
st16 = st16.loc[st16.loc[:, 'last_price'] >= 14990000]
st16 = st16.loc[st16.loc[:, 'last_price'] <= 26037740]

In [ ]:
st17 = flat_data.query('square_type == 17')
st17['last_price'].describe()

In [ ]:
st17 = st17.loc[st17.loc[:, 'last_price'] >= 17500000]
st17 = st17.loc[st17.loc[:, 'last_price'] <= 31300000]

In [ ]:
st18 = flat_data.query('square_type == 18')
st18['last_price'].describe()

In [ ]:
st18 = st18.loc[st18.loc[:, 'last_price'] >= 15965000]
st18 = st18.loc[st18.loc[:, 'last_price'] <= 29887120]

In [ ]:
st19 = flat_data.query('square_type == 19')
st19['last_price'].describe()

In [ ]:
st19 = st19.loc[st19.loc[:, 'last_price'] >= 16249000]
st19 = st19.loc[st19.loc[:, 'last_price'] <= 36576700]

In [ ]:
st20 = flat_data.query('square_type == 20')
st20['last_price'].describe()

In [ ]:
st20 = st20.loc[st20.loc[:, 'last_price'] >= 19275000]
st20 = st20.loc[st20.loc[:, 'last_price'] <= 31679000]

In [ ]:
st888 = flat_data.query('square_type == 888')
st888['last_price'].describe()

In [ ]:
st888 = st888.loc[st888.loc[:, 'last_price'] >= 23125000]
st888 = st888.loc[st888.loc[:, 'last_price'] <= 68000000]

In [ ]:
combine = [st2,st3,st4,st5,st6,st7,st8,st9,st10,st11,st12,st13,st14,st15,st16,st17,st18,st19,st20,st888]
general_df = pd.concat(combine)

In [ ]:
plato1 = general_df.query('(year <= 2016) & (month < 2)')
pic1 = general_df.query('(year == 2016) & (2 < month < 7)')
plato2 = general_df.query('(year == 2016) & (7 <= month <= 12)')
pic2 = general_df.query('(year == 2017) & (4 <= month <= 10)')
mega_pic1 = general_df.query('(year == 2017) & (month == 11)')
plato3_1 = general_df.query('(year == 2017) & (month == 12)')
plato3_2 = general_df.query('(year == 2018) & (month == 1)')
join_plato = [plato3_1, plato3_2]
plato3 = pd.concat(join_plato)
mega_pic2 = general_df.query('(year == 2017) & (month == 11)')
crizis = flat_data.query('(year == 2018) & (4 <= month <= 6)')

In [ ]:
median_price = plato1.pivot_table(index=['square_type'], values='last_price', aggfunc='median')
median_price = median_price.reset_index()
median_price.columns = ['square_type','plato1']

In [ ]:
median_price_pic1 = pic1.pivot_table(index=['square_type'], values='last_price', aggfunc='median')
median_price_pic1 = median_price_pic1.reset_index()
median_price_pic1.columns = ['square_type','pic1']

In [ ]:
median_price_plato2 = plato2.pivot_table(index=['square_type'], values='last_price', aggfunc='median')
median_price_plato2 = median_price_plato2.reset_index()
median_price_plato2.columns = ['square_type','plato2']

median_price_pic2 = pic2.pivot_table(index=['square_type'], values='last_price', aggfunc='median')
median_price_pic2 = median_price_pic2.reset_index()
median_price_pic2.columns = ['square_type','pic2']

median_price_mega_pic1 = mega_pic1.pivot_table(index=['square_type'], values='last_price', aggfunc='median')
median_price_mega_pic1 = median_price_mega_pic1.reset_index()
median_price_mega_pic1.columns = ['square_type','mega_pic1']

median_price_plato3 = plato3.pivot_table(index=['square_type'], values='last_price', aggfunc='median')
median_price_plato3 = median_price_plato3.reset_index()
median_price_plato3.columns = ['square_type','plato3']

median_price_mega_pic2 = mega_pic2.pivot_table(index=['square_type'], values='last_price', aggfunc='median')
median_price_mega_pic2 = median_price_mega_pic2.reset_index()
median_price_mega_pic2.columns = ['square_type','mega_pic2']

median_price_crizis = crizis.pivot_table(index=['square_type'], values='last_price', aggfunc='median')
median_price_crizis = median_price_crizis.reset_index()
median_price_crizis.columns = ['square_type','crizis']

In [ ]:
plato1_pic1 = median_price.merge(median_price_pic1, on='square_type', how='right')
general = plato1_pic1.merge(median_price_plato2, on='square_type', how='right')
general = general.merge(median_price_pic2, on='square_type', how='right')
general = general.merge(median_price_mega_pic1, on='square_type', how='right')
general = general.merge(median_price_plato3, on='square_type', how='right')
general = general.merge(median_price_mega_pic2, on='square_type', how='right')
general = general.merge(median_price_crizis, on='square_type', how='right')
general

In [ ]:
median_price = plato1.pivot_table(index=['square_type'], values='last_price', aggfunc='mean')
median_price = median_price.reset_index()
median_price.columns = ['square_type','plato1']

median_price_pic1 = pic1.pivot_table(index=['square_type'], values='last_price', aggfunc='mean')
median_price_pic1 = median_price_pic1.reset_index()
median_price_pic1.columns = ['square_type','pic1']

median_price_plato2 = plato2.pivot_table(index=['square_type'], values='last_price', aggfunc='mean')
median_price_plato2 = median_price_plato2.reset_index()
median_price_plato2.columns = ['square_type','plato2']

median_price_pic2 = pic2.pivot_table(index=['square_type'], values='last_price', aggfunc='mean')
median_price_pic2 = median_price_pic2.reset_index()
median_price_pic2.columns = ['square_type','pic2']

median_price_mega_pic1 = mega_pic1.pivot_table(index=['square_type'], values='last_price', aggfunc='mean')
median_price_mega_pic1 = median_price_mega_pic1.reset_index()
median_price_mega_pic1.columns = ['square_type','mega_pic1']

median_price_plato3 = plato3.pivot_table(index=['square_type'], values='last_price', aggfunc='mean')
median_price_plato3 = median_price_plato3.reset_index()
median_price_plato3.columns = ['square_type','plato3']

median_price_mega_pic2 = mega_pic2.pivot_table(index=['square_type'], values='last_price', aggfunc='mean')
median_price_mega_pic2 = median_price_mega_pic2.reset_index()
median_price_mega_pic2.columns = ['square_type','mega_pic2']

median_price_crizis = crizis.pivot_table(index=['square_type'], values='last_price', aggfunc='mean')
median_price_crizis = median_price_crizis.reset_index()
median_price_crizis.columns = ['square_type','crizis']

In [ ]:
plato1_pic1 = median_price.merge(median_price_pic1, on='square_type', how='right')
general = plato1_pic1.merge(median_price_plato2, on='square_type', how='right')
general = general.merge(median_price_pic2, on='square_type', how='right')
general = general.merge(median_price_mega_pic1, on='square_type', how='right')
general = general.merge(median_price_plato3, on='square_type', how='right')
general = general.merge(median_price_mega_pic2, on='square_type', how='right')
general = general.merge(median_price_crizis, on='square_type', how='right')
general

In [ ]:
flat_data['square_type'].value_counts()

Я посмотрел на медиану всех данных, посмотрел на медиану данных после исключения "выбивающихся" из общего распределения данных, посмотрел на среднее, которое даёт те же выводы, что и медиана, а именно - особой привязки месяца продажи, количества объявлений на сайте (пик, момент, когда много объявлений или спад) к стоимости нет. Возможно, что это может повлиять на СРОК, ВРЕМЯ продажи. 

Стоит заметить, что стоимость недвижимости увеличивается, если смотреть на года, в годовом плане - стомость увеличивается, возможно тут замешана инфляция, обесценивание денег, увеличение спроса (конечно же за счёт перераспределения населения) при запаздвающем строительстве новой недвижимости. Было бы кстати посмотреть на тип недвижимости - первичная или вторичная.

Давайте посмотрим на распределение по дням недели. Есть предположение, что день публикации не будет влиять на стоимость квартиры, но давайте посмотрим на распределение по дням недели, какие дни люди считают более подходящими для публикации и посмотрим на стоимость по дням недели, возможно, я ошибаюсь, говоря, что день недели не влияет на стоимость.

In [ ]:
flat_data['day'].value_counts()

In [ ]:
general_df['day'].value_counts()

"любимые" дни размещения объявления это - четверг, с небольшим, символическим отрывом идёт вторник. В субботу и воскресенье подают меньше всего объявлений, но тем не менее это примерно в 2,5 раза меньше, чем в четверг, что не так уж и мало.

Давайте посмотрим на самые "дорогие" дни.

In [ ]:
day_price_general = flat_data.pivot_table(index=['day'], values='last_price', aggfunc=['median','mean'])
day_price_general

In [ ]:
day_price_general_camb = general_df.pivot_table(index=['day'], values='last_price', aggfunc=['median','mean'])
day_price_general_camb

In [ ]:
general_df.corr()

Что касается дня недели - наибольшее количество объявлений приходится на средние дни со вторника по пятницу. С субботы до понедельника люди не любят давать объявления. Это и объяснимо, на выходных - люди отдыхают (в большинстве случаев), а понедельник - "день тяжёлый".

##### Выбераю 10 населённых пунктов с наибольшим числом объявлений.

In [ ]:

flat_data['locality_name'].value_counts().head(10)

Далее определяю среднюю цену за квадратный метр в этих населённых пунктов.

##### Санкт-Петербург

In [ ]:
piter = flat_data.query('locality_type == 0')
piter1 = general_df.query('locality_type == 0')

В таблице general_df - я убрал "слишком" дорогие и дешёвые варианты больше в качестве эксперимента, т.к. не считаю слишком дорогую или дешёвую квартиру выбросом, ведь я не знаю что находится внутри этой недвижимости и всех нюансов. Поэтому среднюю стоимость расчитаю также в двух вариантах. Я посчитал среднее и медиану не разделяя по времени подачи объявлений, то есть за все годы, то есть почти за 5 лет, надеюсь это не критично. Среднее выше медианы, из чего можно делать вывод, что квартир с более низкой ценой больше, они представляют большее количество, поэтому считаю нужным работать с медианой.

In [ ]:
mean_piter_price = int(piter['last_price'].sum() / piter['total_area'].sum())
mean_piter_price

In [ ]:
mean_piter_price1 = int(piter1['last_price'].sum() / piter['total_area'].sum())
mean_piter_price1

In [ ]:
median_piter_price = int(piter['last_price'].median() / piter['total_area'].median())
median_piter_price

In [ ]:
median_piter_price1 = int(piter1['last_price'].median() / piter['total_area'].median())
median_piter_price1

In [ ]:
flat_data.info()

In [ ]:
x = int(piter['price_square_meter'].sum() / len(piter))
x

In [ ]:
x = int(piter1['price_square_meter'].sum() / len(piter1))
x

Здесь и далее буду работать с данными из таблицы general_df, откуда я с помощью метода boxplot() убрал выбивающиеся из общего распределения квартиры, т.к. нас больше интересует общая выборка, большинство, а единичные выбивающиеся случаи будут искажать цифру.

Таким образом средняя цена за квадратный метр в Санкт-Питербурге составляет около 100т.р.

##### посёлок Мурино

In [ ]:
murino = flat_data.query('locality_name == "посёлок Мурино"')
murino1 = general_df.query('locality_name == "посёлок Мурино"')


In [ ]:
mean_murino_price = int(murino['last_price'].sum() / murino['total_area'].sum())
mean_murino_price

In [ ]:
mean_murino_price1 = int(murino1['last_price'].sum() / murino1['total_area'].sum())
mean_murino_price1

In [ ]:
x = int(murino['price_square_meter'].sum() / len(murino))
x

In [ ]:
x = int(murino1['price_square_meter'].sum() / len(murino1))
x

Средняя цена за квадратный метр в Мурино составляет 90т.р. Но необходимо заметить, что цена ниже с квартирами, выбивающимися из общей выборки, в отличии от "второй столицы", из этого можно сделать вывод, что в Мурино больше квартир с "занижением" стоимости, то есть людям главное продать свою недвижимость.

##### Шушары

In [ ]:
shush = flat_data.query('locality_name == "посёлок Шушары"')
shush1 = general_df.query('locality_name == "посёлок Шушары"')

In [ ]:
x = int(shush['price_square_meter'].sum() / len(shush))
x

In [ ]:
x = int(shush1['price_square_meter'].sum() / len(shush1))
x

In [ ]:
mean_shush_price1 = int(shush1['last_price'].sum() / shush1['total_area'].sum())
mean_shush_price1

Средняя стоимость квадратного метра в пос.Шушары - 85т.р. И тут наблюдается ситуация схожая с ситуацией в пос.Мурино. Когда имеется тенденция к занижению цены.

##### Всеволожск

In [ ]:
vsk = flat_data.query('locality_name == "Всеволожск"')
vsk1 = general_df.query('locality_name == "Всеволожск"')

In [ ]:
x = int(vsk['price_square_meter'].sum() / len(vsk))
x

In [ ]:
x = int(vsk1['price_square_meter'].sum() / len(vsk1))
x

Средняя стоимость квадратного метра во Всеволожске - 80т.р. Не глядя на карту можно сказать, что Всеволожск не пользуется популярность, небольшой населённый пункт с оттоком населения.

##### Пушкин

In [ ]:
push = flat_data.query('locality_name == "Пушкин"')
push1 = general_df.query('locality_name == "Пушкин"')

In [ ]:
x = int(push['price_square_meter'].sum() / len(push))
x

In [ ]:
x = int(push1['price_square_meter'].sum() / len(push1))
x

А в Пушкине ситуация наоборот, средняя стоимость за кв.м. - 97 т.р., почти как в Спб. И тут имется тенденция к "завышению" цены.

##### Колпино

In [ ]:
kol = flat_data.query('locality_name == "Колпино"')
kol1 = general_df.query('locality_name == "Колпино"')

In [ ]:
x = int(kol['price_square_meter'].sum() / len(kol))
x

In [ ]:
x = int(kol1['price_square_meter'].sum() / len(kol1))
x

Средняя цена за кв.м. - 82 т.р.

##### посёлок Парголово

In [ ]:
par = flat_data.query('locality_name == "посёлок Парголово"')
par1 = general_df.query('locality_name == "посёлок Парголово"')

In [ ]:
x = int(par['price_square_meter'].sum() / len(par))
x

In [ ]:
x = int(par1['price_square_meter'].sum() / len(par1))
x

Средняя цена за кв.м. - 90 т.р. Стало интересно, почему цена немногим уступает цене СПБ и без значительной тенденции к снижению. Заглянул в Википедию - с 2014 года стойкий колоссальный прирост населения. Это вполне объясняет все - спрос резко увеличился.

##### Гатчина

In [ ]:
gat = flat_data.query('locality_name == "Гатчина"')
gat1 = general_df.query('locality_name == "Гатчина"')

In [ ]:
x = int(gat['price_square_meter'].sum() / len(gat))
x

In [ ]:
x = int(gat1['price_square_meter'].sum() / len(gat1))
x

Средняя цена за кв.м. - 82 т.р. Но Гатчину не жалуют, стараются продать и уехать.

##### Кудрово

In [ ]:
kudr01 = flat_data.query('locality_name == "деревня Кудрово"')
kudr02 = flat_data.query('locality_name == "Кудрово"')
kudr11 = general_df.query('locality_name == "деревня Кудрово"')
kudr12 = general_df.query('locality_name == "Кудрово"')
k0 = [kudr01, kudr02]
k1 = [kudr11, kudr12]
kudr = pd.concat(k0)
kudr1 = pd.concat(k1)


In [ ]:
x = int(kudr01['price_square_meter'].sum() / len(kudr01))
x

In [ ]:
x = int(kudr02['price_square_meter'].sum() / len(kudr02))
x

In [ ]:
x = int(kudr['price_square_meter'].sum() / len(kudr))
x

In [ ]:
x = int(kudr1['price_square_meter'].sum() / len(kudr1))
x

In [ ]:
x = int(kudr11['price_square_meter'].sum() / len(kudr11))
x

In [ ]:
x = int(kudr12['price_square_meter'].sum() / len(kudr12))
x

А вот с Кудрово интересная картина. Получив статус города - цена сразу повысилась, почти сравнявшись с ценой в Спб.

##### Выборг

In [ ]:
vbg = flat_data.query('locality_name == "Выборг"')
vbg1 = general_df.query('locality_name == "Выборг"')

In [ ]:
x = int(vbg['price_square_meter'].sum() / len(vbg))
x

In [ ]:
x = int(vbg1['price_square_meter'].sum() / len(vbg1))
x

Средняя цена - 75т.р. Без комментариев.

#### Определяем центр Санкт-Петербурга

In [ ]:
spb = flat_data.query('locality_type == 0')
spb = general_df.query('locality_type == 0')

In [ ]:
spb.dropna(subset=['city_centers_nearest'], inplace=True)
general_df.dropna(subset=['city_centers_nearest'], inplace=True)
spb['city_centers_nearest'] = spb['city_centers_nearest'].astype('int')
general_df['city_centers_nearest'] = general_df['city_centers_nearest'].astype('int')
spb['city_centers_nearest'] = spb['city_centers_nearest'] / 1000
general_df['city_centers_nearest'] = general_df['city_centers_nearest'] / 1000
spb['city_centers_nearest'] = spb['city_centers_nearest'].round(0)
general_df['city_centers_nearest'] = general_df['city_centers_nearest'].round(0)
spb['city_centers_nearest'].value_counts()
grafik = spb[['city_centers_nearest','last_price']]
grafik1 = general_df[['city_centers_nearest','last_price']]
#spb.plot(x='city_centers_nearest', y='last_price', figsize=(10,8), grid=True)

In [ ]:
grafik['city_centers_nearest'].value_counts()

In [ ]:
grafik.plot(x = 'city_centers_nearest', y = 'last_price', style='o', figsize=(15,8), grid=True)

In [ ]:
grafik1.plot(x = 'city_centers_nearest', y = 'last_price', style='o', figsize=(15,8), grid=True)

In [ ]:
every_km = general_df.pivot_table(index=['city_centers_nearest'], values='last_price', aggfunc=['mean','median'])
every_km = every_km.reset_index()
every_km = every_km.drop(42)
every_km

In [ ]:
every_km.plot(x='city_centers_nearest', y='mean',figsize=(15,8), grid=True)

In [ ]:
every_km.plot(x='city_centers_nearest', y='median',figsize=(15,8), grid=True)

Видим, что после третьего километра стоимость начинает снижаться.

#### Изучаем параметры и ценообразующие факторы квартир в центре

In [ ]:
spb = general_df.query('locality_type == 0')
center_spb = spb.query('city_centers_nearest <= 8')
without_center = spb.query('city_centers_nearest > 8')

In [ ]:
center_spb['square_type'].value_counts()

In [ ]:
spb['square_type'].value_counts()

In [ ]:
without_center['square_type'].value_counts()

В центре в продаже превалируют квартиры с большей площадью, причём квартир с площадью более 130 кв.м. - абсолютное большинство по сравнению с "нецентральной" частью города.

In [ ]:
price_center = center_spb.pivot_table(index=['square_type'], values='last_price', aggfunc=['median','mean'])

price_spb = spb.pivot_table(index=['square_type'], values='last_price', aggfunc=['median','mean'])

price_without_center = without_center.pivot_table(index=['square_type'], values='last_price', aggfunc=['median','mean'])

price_center = price_center.reset_index()
price_center.columns = ['square_type','median_center','mean_center']

price_spb = price_spb.reset_index()
price_spb.columns = ['square_type','median_spb','mean_spb']

price_without_center = price_without_center.reset_index()
price_without_center.columns = ['square_type','median_with_centr','mean_with_centr']

prices0 = price_center.merge(price_spb, on='square_type', how='right')
prices = prices0.merge(price_without_center, on='square_type', how='right')
prices

In [ ]:
difference = prices[['square_type']]
difference['median_center/spb'] = prices['median_center'] - prices['median_spb']
difference['mean_center/spb'] = prices['mean_center'] - prices['mean_spb']
difference['median_center/without_centr'] = prices['median_center'] - prices['median_with_centr']
difference['mean_center/without_centr'] = prices['mean_center'] - prices['mean_with_centr']
difference['median_spb/without_centr'] = prices['median_spb'] - prices['median_with_centr']
difference['mean_spb/without_centr'] = prices['mean_spb'] - prices['mean_with_centr']


difference

In [ ]:
spb.query('9 < square_type < 13')

Мы вновь убедились, что квартиры в общей сложности дороже в центре, увидели разницу - от 50 тысяч до  нескольких миллионов. Но есть аномалия среди квартир площадью 90 - 120 кв.м., для них общие правила не действуют, как медиана, так и среднее выше за пределами центра. С чем это может быть связано? Возможно, что это связано с годом публикации объявления и большинство квартир за пределами условного центра выставлены в более поздние сроки, когда инфляция сделала своё. Давайте проверим это.

In [ ]:
anomalia_spb = spb.query('9 < square_type < 13')
years_square = anomalia_spb.pivot_table(index=['year', 'square_type'], values='total_area', aggfunc='count')
years_square = years_square.reset_index()
years_square.columns = ['year','square_type','count_spb']

anomalia_center = center_spb.query('9 < square_type < 13')
years_square_center = anomalia_center.pivot_table(index=['year', 'square_type'], values='total_area', aggfunc='count')
years_square_center = years_square_center.reset_index()
years_square_center.columns = ['year','square_type','count_center']
years_square['count_center'] = years_square_center['count_center']


years_square

In [ ]:
spb2018 = spb.query('year == 2018')
center2018 = spb2018.query('city_centers_nearest <= 8')
without_center2018 = spb2018.query('city_centers_nearest > 8')

price_center = center2018.pivot_table(index=['square_type'], values='last_price', aggfunc=['median','mean'])

price_spb = spb2018.pivot_table(index=['square_type'], values='last_price', aggfunc=['median','mean'])

price_without_center2018 = without_center2018.pivot_table(index=['square_type'], values='last_price', aggfunc=['median','mean'])

price_center = price_center.reset_index()
price_center.columns = ['square_type','median_center','mean_center']

price_spb = price_spb.reset_index()
price_spb.columns = ['square_type','median_spb','mean_spb']

price_without_center2018 = price_without_center2018.reset_index()
price_without_center2018.columns = ['square_type','median_with_centr','mean_with_centr']

prices0 = price_center.merge(price_spb, on='square_type', how='right')
prices = prices0.merge(price_without_center2018, on='square_type', how='right')
prices

Я взял только 2018 год и картина не изменилась. Значит год не влияет. Возможно спрос на данную площадь выше, что подогревает рост стоимости.

Посмотрим как количество комнат распределено по центру и по городу в целом. Т.к. мы уже знаем, что количество комнат прямо пропорционально площади квартиры, то ожидаемый результат бужет аналогичен распределению площади в зависимости от центра к городу.

In [ ]:
center_spb['rooms'].value_counts()

In [ ]:
spb['rooms'].value_counts()

In [ ]:
without_center['rooms'].value_counts()

In [ ]:
center_spb.plot(x='rooms', y='last_price', style='o', figsize=(15,8), grid=True)

In [ ]:
spb.plot(x='rooms', y='last_price', style='o', figsize=(15,8), grid=True)

In [ ]:
without_center.plot(x='rooms', y='last_price', style='o', figsize=(15,8), grid=True)

Мы убедились, что самые большие квартиры расположены в центральной части, которая является и чемпионом по многокомнатности.

In [ ]:
center_spb['ceiling_height'].hist(bins=50, figsize=(10,8))

In [ ]:
without_center['ceiling_height'].hist(bins=50, figsize=(10,8))

Да, квартиры в центральной части города не только больше по площади, но и выше. 

In [ ]:
center_spb['floor'].value_counts()

Сразу видно, что преобладают пяти-этажные здания.

In [ ]:
spb['floor'].value_counts()

In [ ]:
center_spb['floor_type'].hist()

In [ ]:
spb['floor_type'].hist()

Цетральная часть застроена пятиэтажными домами, поэтому превалирует продажа от 1 до 5 этажа, с единичными случаями выше 5го. По всему городу до 10 этажа - превалирующие предложения. И в подавляющем большинстве это не 1 и последний этажи. Влияние этажа на стоимость имеется, но не настолько критичный как фактор площади и близости к центру.

Относительно даты подачи объявлений - также пришли в прошлых этапах анализа к выводу, что с годом стоимость растёт, по месяцам, по количеству объявлений - продающие не смотрят, когда определяют стоимость своей квартиры. 2019 год представлен не полностью, максимум объявлений приходится на 2018 год.

### Шаг 5. Общий вывод


В ходе работы стали известны параметры, которые влияют на стоимость квартиры. Это в первую очередь - площадь (количество комнат не рассматриваю, т.к. они коррелируют с площадью), близость к центру, в меньшей мере - этаж. Высота потолков - больше в зданиях ранней постройки, расположенных в центре, следует заметить, что в зданиях в центре и потолки выше и площадь больше, что также сказывается на росте стоимости.

Относительно даты публикации объявления - можно сказать, что с инфляцией растет и цена 